### Реализовать на С или python поиск минимального значения для каждой из 5 заданных двумерных функций f(x,y):
(a) fa(x,y)=(x−3)^2+y^2+1

(b) fb(x,y)=(x+1)^2+(2y−8)^2+1

(c) fc(x,y)=x^2+xy+y^2−16x−17y+94

(d) fd(x,y)=((x−7)^2+(y−8)^2)·((x−8)^2+(y−7)^2)+4

(e) fe(x,y)=(4xy−19)2·(cos2πx+cos2πy)−x−y+24

Решение вычислить с точностью ε = 10−6.

Найти точку минимума (x∗,y∗), значение функции f(x∗,y∗) и количество итераций (вычислений функции) n, которое потребовалось на достижение требуемой точности выбранным методом.

In [32]:
import math
import numpy as np

globalEpsilon = 1e-6
radius = 10                                 # working plane radius
center = (0.1, 2)                          # center of the working circle
arrShape = 100                             # number of points processed / 360
step = radius / arrShape                   # step between two points
iterator = 0
descentStepSpeed = 0.95


# определяем функции, для которых будем считать минимум градиентным спуском:
def firstFunction(x, y):
    return (x - 3) * (x - 3)  + y * y + 1

def secondFunction(x, y):
    return (x + 1)*(x + 1) + (2*y - 8)*(2*y - 8) + 1

def thirdFunction(x, y):
    return x * x + (x * y) + y * y - (16 * x) - (17 * y) + 94

def fourthFunction(x, y):
    return ((x - 7) * (x - 7) + (y - 8) * (y - 8)) * ((x - 8) * (x - 8) + (y - 7) * (y - 7)) + 4

def fifthFunction(x, y):
    return (4 * x * y - 19) * (4 * x * y - 19) * (math.cos(2 * x * math.pi) + math.cos(2 * y * math.pi)) - x - y+24

# определяем вспомогательные функции для градиентного спуска:
def rotateVector(length, a):
    return length * np.cos(a), length * np.sin(a)

def getFlipPoints(customFunction):
    flipPoints = np.array([[0, 0], [0, 0]])
    points = np.zeros((360, arrShape), dtype=bool)
    cx, cy = center

    for i in range(arrShape):
        for alpha in range(360):
            x, y = rotateVector(step, alpha)
            x = x * i + cx
            y = y * i + cy
            points[alpha][i] = derivativeByX(x, y, customFunction) + derivativeByY(y, x, customFunction) > 0
            if not points[alpha][i - 1] and points[alpha][i]:
                flipPoints = np.vstack((flipPoints, np.array([alpha, i - 1])))

    return flipPoints

def getEstimates(positions, customFunction):
    vx, vy = rotateVector(step, positions[1][0])
    cx, cy = center
    bestX, bestY = cx + vx * positions[1][1], cy + vy * positions[1][1]

    for index in range(2, len(positions)):
        vx, vy = rotateVector(step, positions[index][0])
        x, y = cx + vx * positions[index][1], cy + vy * positions[index][1]
        if customFunction(bestX, bestY) > customFunction(x, y):
            bestX = x
            bestY = y

    for index in range(360):
        vx, vy = rotateVector(step, index)
        x, y = cx + vx * (arrShape - 1), cy + vy * (arrShape - 1)
        if customFunction(bestX, bestY) > customFunction(x, y):
            bestX = x
            bestY = y

    return bestX, bestY

def derivativeByY(epsilon, arg, customFunction):
    return (customFunction(arg, epsilon + globalEpsilon) - customFunction(arg, epsilon)) / globalEpsilon

def derivativeByX(epsilon, arg, customFunction):
    return (customFunction(globalEpsilon + epsilon, arg) - customFunction(epsilon, arg)) / globalEpsilon

def gradient(x, y, customFunction):
    return derivativeByX(x, y, customFunction) + derivativeByY(y, x, customFunction)

def gradientDescent(bestEstimates, isX, customFunction):
    derivative = derivativeByX if isX else derivativeByY
    bestX, bestY = bestEstimates
    descentStep = step
    global iterator
    value = derivative(bestY, bestX, customFunction)

    while abs(value) > abs(globalEpsilon):
        # print("value = {val}, globalEpsilon = {eps}".format(val = value, eps = globalEpsilon ))
        iterator += 1
        descentStep *= descentStepSpeed
        bestY = bestY - descentStep \
            if derivative(bestY, bestX, customFunction) > 0 else bestY + descentStep
        value = derivative(bestY, bestX, customFunction)
    return bestY, bestX

def findMinimum(customFunction):
    return gradientDescent(gradientDescent(getEstimates(getFlipPoints(customFunction), customFunction), False, customFunction), True, customFunction)

In [10]:
def printCalculationData(myFunction, name):
    global iterator
    minX, minY = findMinimum(myFunction)
    minimum = (minX, minY)
    functionMinValue = myFunction(minX, minY)
    print("{name} function min = ({minX}, {minY}), function value = {val} iterations = {i}".format(name = name, minX = minX, minY = minY, val = functionMinValue, i = iterator))
    iterator = 0

In [33]:
# рассчитаем минимумы для пяти функций:
printCalculationData(firstFunction, "First")
printCalculationData(secondFunction, "Second")
printCalculationData(thirdFunction, "Third")
printCalculationData(fourthFunction, "Fourth")
printCalculationData(fifthFunction, "Fifth")

First function min = (2.9999995855684203, -5.653044802199055e-07), function value = 1.0000000000004914 iterations = 192
Second function min = (-1.0000004882595799, 3.9999995488837756), function value = 1.0000000000010525 iterations = 158
Third function min = (4.989420924360018, 6.021157538996236), function value = 3.0003357310923775 iterations = 124
Fourth function min = (6.9999999126671195, 7.999249948935995), function value = 4.000001124309705 iterations = 154
Fifth function min = (-7.512534507372659, 8.510398559535755), function value = -150544.63132472942 iterations = 652
